## TensorFlow 는 구글에서 만든 머신러닝 라이브러리

## <ol>tf.keras.layers</ol>

In [1]:
import tensorflow as tf

In [2]:
# 텐서플로우 버전확인
tf.__version__

'2.10.0'

## GPU 설정! 

In [22]:
print("Num GPUs Available: ", 
len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
%%time
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

tf.enable_v2_behavior()

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()


(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)


In [3]:
INPUT_SIZE = (20,1)

inputs = tf.keras.layers.Input(shape=INPUT_SIZE)
output = tf.keras.layers.Dense(units=10,activation=tf.nn.sigmoid)(inputs)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-02 13:55:10.337401: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-02 13:55:10.337526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
INPUT_SIZE = (20,1)

inputs = tf.keras.layers.Input(shape=INPUT_SIZE)
hidden = tf.keras.layers.Dense(units=10,activation=tf.nn.sigmoid)(inputs)
output = tf.keras.layers.Dense(units=2, activation=tf.nn.sigmoid)(hidden)


## tf.keras.layers.Dropout 

In [5]:
#신경망 모델을 만들 때 생기는 문제점 중 대표적인 문제점으로 과적합(Overfitting)이 있다
#과적합 문제는 정규화(Regularization)방법을 사용해서 해결하는데, 가장 대표적인 방법이 드롭아웃(Dropout)이다.

INPUT_SIZE = (20,1)

inputs = tf.keras.layers.Input(shape=INPUT_SIZE)
#Dropout의 rate=0.5는 전체 입력값중에 50%를 0으로 만든다는 뜻
dropout = tf.keras.layers.Dropout(rate=0.5)(inputs)


In [6]:
# Dropout은 tf.keras.layers뿐만 아니라 tf.nn모델에도 있는데 
# 두 모듈의 차이점은 tf.keras.layers.Dropout의 경우 확률을 0.2라 했을때 20%를 0으로 만들지만
# tf.nn.dropout은 0.2라 했을때 80%를 0으로 만든다.

INPUT_SIZE = (20 , 1)

inputs = tf.keras.layers.Input(shape=INPUT_SIZE)
dropout = tf.keras.layers.Dropout(rate=0.2)(inputs)
hidden = tf.keras.layers.Dense(units=10,activation=tf.nn.sigmoid)(dropout)
output = tf.keras.layers.Dense(units=2, activation=tf.nn.sigmoid)(hidden)


## tf.keras.layers.Conv1D

In [7]:
# 합성곱 연산중 Conv1D를 알아보자
# 책의 24page 참조
# Conv1D의 기본사용법

INPUT_SIZE = (1,28,28)

inputs = tf.keras.Input(shape=INPUT_SIZE)
conv = tf.keras.layers.Conv1D(
    filters=10,#필터의 개수로, 정수형으로 지정한다. 출력의 차원수를 나타낸다.
    kernel_size=3, #필터의 크기로서, 정수 혹은 정수의 리스트, 튜플 형태로 지정한다. 합성곱이 적용되는 윈도의 길이를 나타낸다.
    padding='same', #패딩 방법을 지정한다. 'VALID'혹은 'SAME'으로 지정가능
    activation=tf.nn.relu #활성화 함수
)(inputs)


In [8]:
# 입력값의 드롭아웃을 적용한 합성곱 신경망도 Dropout과 Conv1D를 사용해서 구현할 수 있다.

INPUT_SIZE = (1,28,28)

inputs=tf.keras.Input(shape= INPUT_SIZE)
dropout = tf.keras.layers.Dropout(rate=0.2)(inputs)
conv = tf.keras.layers.Conv1D(
    filters=10,
    kernel_size = 3,
    padding='same',
    activation = tf.nn.relu)(dropout)

## tf.keras.layers.MaxPool1D

In [9]:
# tf.keras.layers.MaxPool1D 는 합성곱 신경망과 함께 쓰이는 기법 중 하나
# 피처 맵크기를 줄이거나 주요한 특징을 뽑아내기 위해 합성곱 이후에 적용되는 기법

INPUT_SIZE = (1,28,28)

inputs = tf.keras.Input(shape=INPUT_SIZE)
dropouts = tf.keras.layers.Dropout(rate=0.2)(inputs)
conv = tf.keras.layers.Conv1D(
    filters=10,
    kernel_size=3,
    padding='same',
    activation=tf.nn.relu)(dropouts)
max_pool = tf.keras.layers.MaxPool1D(pool_size=3, padding='same')(conv)
flatten = tf.keras.layers.Flatten()(max_pool) # Flatten 은 맥스 풀링 결괏값을 완전 연결 계층으로 연결하기 위해서는 행렬이었던 것을 벡터로 만들어준다.
hidden = tf.keras.layers.Dense(units=50, activation=tf.nn.relu)(flatten)
output = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)(hidden)

ValueError: Input 0 of layer "max_pooling1d" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 1, 28, 10)

## 모델구축

<li> Sequential API</li>
<li>Functional API</li>
<li>Functional / Sequential API</li><ol>+ Custom Layers</ol>    
<li>Sequential API(Custom Model)</li>

## Sequential API

<li> 케라스를 활용해 모델을 구축하는데 가장 간단한 API</li>
<li> 하나의 플로만 계산할 수 있는 Sequentioal은 두 개의 값을 합칠 수가 없어서 여러 제약이 존재</li>
<ol>
    <li>다중 입력값 모델(Multi-input models)</li>
    <li>다중 출력값 모델(Multi-output models)</li>
    <li>공유 층을 활용하는 모델(Models with shared layers)</li>
    <li>데이터 흐름이 순차적이지 않은 모델(Models with non-sequential data flows)</li>
</ol>
<li>에서는 여러 제약이 존재</li>

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=10, activation='softmax'))


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-02 14:12:35.553467: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-02 14:12:35.553596: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Functional API
<ol>
    <li>다중 입력값 모델(Multi-input models)</li>
    <li>다중 출력값 모델(Multi-output models)</li>
    <li>공유 층을 활용하는 모델(Models with shared layers)</li>
    <li>데이터 흐름이 순차적이지 않은 모델(Models with non-sequential data flows)</li>
</ol>
 
<li></li>

In [4]:
inputs = tf.keras.Input(shape=(32,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

## Custom Layer

<li>Sequential API 와 Functional API 를 사용하기 위해 케라스의 layers 패키지에 정의된 레이어를 사용해 구현했다</li>
<li>새로운 연산을 하는 레이어 or 편의를 위해 여러 레이어를 하나로 묶은 레이어를 구현해야하는 경우가 있다</li>
<li>이때 사용자 정의 층(custom layer)을 만들어 사용하면 된다.</li>

In [5]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, hidden_dimension2, output_dimension):
        self.hidden_dimension = hidden_dimension
        self.hidden_dimension2 = hidden_dimension2
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__()

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.hidden_dimension2, activation = 'relu')
        self.dense_layer3 = layers.Dense(self.output_dimension, activation = 'softmax')

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        hidden_output = self.dense_layer2(hidden_output)
        return self.dense_layer2(hidden_output)

In [6]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(CustomLayer(64,64,10))

## Subclassing (Custom Model)



In [7]:
class MyModel(tf.keras.Model):

    def __init__(self,hidden_dimension, hidden_dimension2, output_dimension):
        super(MyModel, self).__init__(name='my_model')
        self.dense_layer1 = layers.Dense(hidden_dimension, activation='relu')
        self.dense_layer2 = layers.Dense(hidden_dimension2, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='softmax')

    def call(self, inputs):
        x = self.dense_layer1(inputs)
        x = self.dense_layer2(x)
        
        return self.dense_layer3

## 이제 본격적으로 모델을 구현해 보자!

In [8]:
#텍스트 데이터를 토큰화하기!

import tensorflow as tf
import keras
from tensorflow.keras import preprocessing
from tensorflow.keras import layers
import numpy as np

samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

labels = [[1],[0],[1],[1],[0],[1]]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
print(sequences)

input_sequences = np.array(sequences)
print(input_sequences)

labels = np.array(labels)

word_index = tokenizer.word_index
print(word_index)

[[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]
[[ 4  1  5  6]
 [ 7  1  8  9]
 [10  2  3 11]
 [12  2  3 13]
 [14  1 15 16]
 [17 18 19 20]]
{'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}


In [9]:
# 추가로 모델 구축 및 모델 학습에 필요한 변수 정의

batch_size = 2
num_epochs = 100
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

## Keras Sequential

In [10]:
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length=4))
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(layers.Dense(hidden_dimension, activation='relu'))
model.add(layers.Dense(output_dimension, activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 128)            2688      
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
            loss='binary_crossentropy',
            metrics=['accuracy'])

In [13]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100


2022-11-02 14:12:44.925960: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-02 14:12:45.291821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 18ms/step - loss: 0.6950 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 10ms/step - loss: 0.6759 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: 0.6611 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: 0.6458 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 11ms/step - loss: 0.6272 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 11ms/step - loss: 0.6091 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 12ms/step - loss: 0.5851 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 11ms/step - loss: 0.5570 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 11ms/step - loss: 0.5255 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 11ms/step - loss: 0.4873 - accuracy: 1.0000
Epoch 11/100
3/3 [=============

## Keras Functional API

In [14]:
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [15]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4)]               0         
                                                                 
 embedding_1 (Embedding)     (None, 4, 128)            2688      
                                                                 
 tf.math.reduce_mean (TFOpLa  (None, 128)              0         
 mbda)                                                           
                                                                 
 dense_8 (Dense)             (None, 256)               33024     
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________

In [17]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 11ms/step - loss: 0.6932 - accuracy: 0.5000


2022-11-02 14:12:52.144715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/100
3/3 [==============================] - 0s 15ms/step - loss: 0.6729 - accuracy: 0.6667
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: 0.6537 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 11ms/step - loss: 0.6351 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 12ms/step - loss: 0.6137 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 11ms/step - loss: 0.5901 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5607 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 11ms/step - loss: 0.5279 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 10ms/step - loss: 0.4898 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 10ms/step - loss: 0.4478 - accuracy: 1.0000
Epoch 11/100
3/3 [==============================] - 0s 10ms/step - loss: 0.4021 - accuracy: 1.0000
Epoch 12/100
3/3 [

## Keras Custom Model

In [18]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [19]:
model = CustomModel(vocab_size = vocab_size,
            embed_dimension=emb_size,
            hidden_dimension=hidden_dimension,
            output_dimension=output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100


: 

: 

In [6]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, output_dimension, **kwargs):
        self.hidden_dimension = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.output_dimension)

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        return self.dense_layer2(hidden_output)

    # Optional
    def get_config(self):
        base_config = super(CustomLayer, self).get_config()
        base_config['hidden_dim'] = self.hidden_dimension
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [9]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length = 4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
    CustomLayer(hidden_dimension, output_dimension),
    layers.Activation('sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Train on 6 samples
Epoch 1/100


2022-11-02 14:07:40.264224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-02 14:07:40.296582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-02 14:07:40.328102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-02 14:07:40.341361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 46ms/sample - loss: 0.6937 - accuracy: 0.5000
Epoch 2/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.6749 - accuracy: 0.6667
Epoch 3/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.6586 - accuracy: 0.8333
Epoch 4/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.6427 - accuracy: 0.8333
Epoch 5/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.6243 - accuracy: 0.8333
Epoch 6/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.6027 - accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.5756 - accuracy: 1.0000
Epoch 8/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.5489 - accuracy: 1.0000
Epoch 9/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.5127 - accuracy: 1.0000
Epoch 10/100
6/6 [==============================] - 0s 5ms/sample - loss: 0.4735 - accuracy: 1.0000
Epoch 11/100
6/6 [==